# Importing Libraries

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

# Reading Docs

In [6]:
path = "Docs/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))
    
for f in files:
    print(f)

Docs/file1
Docs/file2
Docs/file3.txt


# Removing Headers and HTML Tags

In [8]:
for f in files:
    doc = open(f,"r")
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    print("Doc : " + text + "\n\n")

doc 1 : Dark Chocolate's Health Benefits
Chocolate and Your Health: The Basics
The Basics
It's more than wishful thinking — chocolate can be good for you.
Studies show that eating chocolate, primarily dark chocolate, may contribute to improved cardiovascular health. Packed with natural antioxidants, dark chocolate and cocoa sit in the same good-for-you category as green tea and blueberries. That's because chocolate comes from cacao beans (or cocoa beans), which grow on the cacao tree and are full of natural plant nutrients. Most of the studies to date highlight dark chocolate's health values because it has the highest percentage of cocoa solids, therefore more flavanol antioxidants.
An Antioxidant Powerhouse
Dark chocolate and cocoa are rich in cell-protecting antioxidants -- natural compounds found in fruits, vegetables, grains and nuts. Read what scientists are studying regarding antioxidants and cardiovascular disease, premature aging and cancer.
Matters of the Heart
Recent studies 